In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.4.0
sys.version_info(major=3, minor=7, micro=9, releaselevel='final', serial=0)
matplotlib 3.3.3
numpy 1.19.2
pandas 1.2.0
sklearn 0.24.0
tensorflow 2.4.0
tensorflow.keras 2.4.0


In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(xtrain_all, ytrain_all), (xtest, ytest) = fashion_mnist.load_data()
xvalid, xtrain = xtrain_all[:5000], xtrain_all[5000:]
yvalid, ytrain = ytrain_all[:5000], ytrain_all[5000:]

print(xvalid.shape, yvalid.shape)
print(xtrain.shape, ytrain.shape)
print(xtest.shape, ytest.shape)

In [ ]:
print(np.max(xtrain), np.min(xtrain))

In [ ]:
# Normalization
# x = (x - u) / std
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# xtrain [None, 28, 28] => [None, 784]
xtrain_scaled = scaler.fit_transform(
    xtrain.astype(np.float32).reshape(-1, 1)).reshape(-1, 28 * 28)
xvalid_scaled = scaler.transform(
    xvalid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28 * 28)
xtest_scaled = scaler.transform(
    xtest.astype(np.float32).reshape(-1, 1)).reshape(-1, 28 * 28)

y_train = np.asarray(y_train, dtype = np.int64)
y_valid = np.asarray(y_valid, dtype = np.int64)
y_test = np.asarray(y_test, dtype = np.int64)

In [ ]:
print(np.max(xtrain_scaled), np.min(xtrain_scaled))

### 在归一化后构建 dataset

In [ ]:
def make_dataset(images, labels, epochs, batch_size, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [ ]:
batch_size = 20
epochs = 10

images_placeholder = tf.placeholder(tf.float32, [None, 28 * 28])
labels_placeholder = tf.placeholder(tf.int64, (None,))

dataset = make_dataset(images_placeholder, labels_placeholder,
                       epochs = epochs,
                       batch_size = batch_size)

# make_initializable_iterator 可以重复初始化，这样数据是变化的
dataset_iter = dataset.make_initializable_iterator()
x, y = dataset_iter.get_next()
with tf.Session() as sess:
    sess.run(dataset_iter.initializer,
             feed_dict = {
                 images_placeholder: x_train_scaled,
                 labels_placeholder: y_train
             })
    x_val, y_val = sess.run([x, y])
    print(x_val.shape)
    print(y_val.shape)
    sess.run(dataset_iter.initializer,
             feed_dict = {
                 images_placeholder: x_valid_scaled,
                 labels_placeholder: y_valid,
             })
    x_val, y_val = sess.run([x, y])
    print(x_val.shape)
    print(y_val.shape)